In [1]:
from profiler.core import *

In [2]:
pf = Profiler(workers=2, verbose=False, tol=0.01, eps=0.1)

In [3]:
#pf.session.load_data(name='flights', src='file', fpath='/Users/zhihan/Documents/bart/data/clean_data/flights.csv', embedtxt=False)
pf.session.load_data(name='hospital', src='file', fpath='data/hospital.csv', embedtxt=True)

INFO:profiler.utility:[0.007055044174194336] Load Data start

INFO:profiler.dataset:inferred types of attributes: {
    "ProviderNumber": "text",
    "HospitalName": "text",
    "Address1": "text",
    "Address2": "categorical",
    "Address3": "categorical",
    "City": "text",
    "State": "categorical",
    "ZipCode": "text",
    "CountyName": "text",
    "PhoneNumber": "text",
    "HospitalType": "text",
    "HospitalOwner": "text",
    "EmergencyService": "categorical",
    "Condition": "text",
    "MeasureCode": "text",
    "MeasureName": "text",
    "Score": "text",
    "Sample": "text",
    "Stateavg": "text"
}
INFO:profiler.dataset:(possible types: numeric, categorical, text, date)
INFO:profiler.dataset:inferred operators of attributes: {'ProviderNumber': ['equal'], 'HospitalName': ['equal'], 'Address1': ['equal'], 'Address2': ['equal'], 'Address3': ['equal'], 'City': ['equal'], 'State': ['equal'], 'ZipCode': ['equal'], 'CountyName': ['equal'], 'PhoneNumber': ['equal'], 'Hospi

In [4]:
pf.session.change_dtypes(['ProviderNumber', 'ZipCode', 'PhoneNumber', 'State', 'EmergencyService','Score', 'Sample'], 
                            [CATEGORICAL, NUMERIC, CATEGORICAL, TEXT, TEXT, NUMERIC, NUMERIC],
                            [None, None, None, None, None, r'(\d+)%', r'(\d+)\spatients'])

INFO:profiler.dataset:updated types of ProviderNumber to 'categorical'
INFO:profiler.dataset:updated operators of ProviderNumber to ['equal']
INFO:profiler.dataset:updated types of ZipCode to 'numeric'
INFO:profiler.dataset:updated operators of ZipCode to ['equal']
INFO:profiler.dataset:updated types of PhoneNumber to 'categorical'
INFO:profiler.dataset:updated operators of PhoneNumber to ['equal']
INFO:profiler.dataset:updated types of State to 'text'
INFO:profiler.dataset:updated operators of State to ['equal']
INFO:profiler.dataset:updated types of EmergencyService to 'text'
INFO:profiler.dataset:updated operators of EmergencyService to ['equal']
INFO:profiler.dataset:updated types of Score to 'numeric'
INFO:profiler.dataset:updated operators of Score to ['equal']
INFO:profiler.dataset:updated types of Sample to 'numeric'
INFO:profiler.dataset:updated operators of Sample to ['equal']
INFO:profiler.dataset:updated inferred operators of attributes: {'ProviderNumber': ['equal'], 'Hospi

In [5]:
pf.session.load_embedding(save=True, path='data/hospital/', load=True)

INFO:profiler.utility:[0.11103606224060059] Load Embedding start

INFO:profiler.utility:[0.14326214790344238] Load Embedding execution time: 0.0322260856628418



In [6]:
pf.session.load_training_data()

INFO:profiler.utility:[0.14936590194702148] Create Training Data start

INFO:profiler.transformer:needs multiplier = 54 to bound the error in inv cov estimation <= 0.10000000
INFO:profiler.transformer:use multiplier = 54, and the bound is 0.00024939
INFO:profiler.transformer:Draw Pairs
100%|██████████| 19/19 [00:00<00:00, 247.44it/s]
INFO:profiler.transformer:Number of training samples: 56943
INFO:profiler.transformer:Computing Differences
INFO:profiler.transformer:estimated missing data probability in training data is 0.0283
INFO:profiler.utility:[0.6679620742797852] Create Training Data execution time: 0.5185961723327637



In [ ]:
NTDs = pf.session.learn_structure(sparsity=0.05)

INFO:profiler.utility:[0.675926923751831] Learn Structure start

INFO:profiler.learner:use threshold 0.0829


<Figure size 1000x800 with 2 Axes>

ProviderNumber_eq,Address1_eq,City_eq,ZipCode_eq,CountyName_eq,PhoneNumber_eq,HospitalOwner_eq,EmergencyService_eq -> HospitalName_eq
ProviderNumber_eq,HospitalName_eq,City_eq,ZipCode_eq,CountyName_eq,PhoneNumber_eq,HospitalOwner_eq -> Address1_eq
ProviderNumber_eq,HospitalName_eq,Address1_eq,ZipCode_eq,CountyName_eq,PhoneNumber_eq,HospitalOwner_eq -> City_eq
ProviderNumber_eq,HospitalName_eq,Address1_eq,City_eq,CountyName_eq,PhoneNumber_eq -> ZipCode_eq
ProviderNumber_eq,HospitalName_eq,Address1_eq,City_eq,ZipCode_eq,PhoneNumber_eq,HospitalOwner_eq -> CountyName_eq
ProviderNumber_eq,HospitalName_eq,Address1_eq,City_eq,ZipCode_eq,CountyName_eq,HospitalOwner_eq -> PhoneNumber_eq
ProviderNumber_eq,HospitalName_eq,Address1_eq,City_eq,CountyName_eq,PhoneNumber_eq -> HospitalOwner_eq
HospitalName_eq -> EmergencyService_eq
MeasureCode_eq,MeasureName_eq -> Condition_eq
Condition_eq,MeasureName_eq,Stateavg_eq -> MeasureCode_eq
Condition_eq,MeasureCode_eq,Stateavg_eq -> MeasureName_eq
MeasureCo

<Figure size 1200x600 with 1 Axes>

<Figure size 1200x600 with 1 Axes>

...

<Figure size 1200x600 with 1 Axes>

<Figure size 1200x600 with 1 Axes>

DEBUG:profiler.learner:check node t = 3 with X(t) = frozenset() 
DEBUG:profiler.learner:check node t = 4 with X(t) = frozenset({1}) 
DEBUG:profiler.learner:check node t = 2 with X(t) = frozenset({1, 10}) 
DEBUG:profiler.learner:check node t = 5 with X(t) = frozenset({1}) 
DEBUG:profiler.learner:check node t = 6 with X(t) = frozenset({0, 1}) 
DEBUG:profiler.learner:check node t = 7 with X(t) = frozenset({0, 1, 2}) 
DEBUG:profiler.learner:check node t = 8 with X(t) = frozenset({0, 1, 2, 3}) 
DEBUG:profiler.learner:check node t = 9 with X(t) = frozenset({0, 1, 2, 3, 6}) 
DEBUG:profiler.learner:check node t = 10 with X(t) = frozenset({0, 1, 2, 3, 6, 7}) 
DEBUG:profiler.learner:check node t = 0 with X(t) = frozenset({0, 1, 2, 3, 6, 7, 9}) 
DEBUG:profiler.learner:check node t = 11 with X(t) = frozenset({0, 1, 2, 3, 6, 7}) 
DEBUG:profiler.learner:check node t = 1 with X(t) = frozenset({0, 1, 2, 3, 5, 6, 7}) 
DEBUG:profiler.learner:check node t = 12 with X(t) = frozenset({1, 2, 3, 5, 6, 7}) 
D

[3]forget:frozenset()
--[4]forget:frozenset({1})
----[2]introduce:frozenset({1, 10})
------[5]forget:frozenset({1})
--------[6]forget:frozenset({0, 1})
----------[7]forget:frozenset({0, 1, 2})
------------[8]forget:frozenset({0, 1, 2, 3})
--------------[9]forget:frozenset({0, 1, 2, 3, 6})
----------------[10]forget:frozenset({0, 1, 2, 3, 6, 7})
------------------[0]introduce:frozenset({0, 1, 2, 3, 6, 7, 9})
--------------------[11]forget:frozenset({0, 1, 2, 3, 6, 7})
----------------------[1]introduce:frozenset({0, 1, 2, 3, 5, 6, 7})
------------------------[12]introduce:frozenset({1, 2, 3, 5, 6, 7})
--------------------------[13]introduce:frozenset({2, 3, 5, 6, 7})
----------------------------[14]introduce:frozenset({3, 5, 6, 7})
------------------------------[15]introduce:frozenset({5, 6, 7})
--------------------------------[16]introduce:frozenset({6, 7})
----------------------------------[17]leaf:frozenset({7})


DEBUG:profiler.learner:check node t = 14 with X(t) = frozenset({3, 5, 6, 7}) 
DEBUG:profiler.learner:check node t = 15 with X(t) = frozenset({5, 6, 7}) 
DEBUG:profiler.learner:check node t = 16 with X(t) = frozenset({6, 7}) 
DEBUG:profiler.learner:R for leaf node t = 17 with X(t) = frozenset({7}) candidate size: 128
DEBUG:profiler.learner:[({7: {0}}, {}, 0), ({7: {1}}, {}, 0), ({7: {2}}, {}, 0), ({7: {3}}, {}, 0), ({7: {5}}, {}, 0), ({7: {6}}, {}, 0), ({7: {9}}, {}, 0), ({7: {0, 1}}, {}, 0), ({7: {0, 2}}, {}, 0), ({7: {0, 3}}, {}, 0), ({7: {0, 5}}, {}, 0), ({7: {0, 6}}, {}, 0), ({7: {0, 9}}, {}, 0), ({7: {1, 2}}, {}, 0), ({7: {1, 3}}, {}, 0), ({7: {1, 5}}, {}, 0), ({7: {1, 6}}, {}, 0), ({7: {1, 9}}, {}, 0), ({7: {2, 3}}, {}, 0), ({7: {2, 5}}, {}, 0), ({7: {2, 6}}, {}, 0), ({7: {9, 2}}, {}, 0), ({7: {3, 5}}, {}, 0), ({7: {3, 6}}, {}, 0), ({7: {9, 3}}, {}, 0), ({7: {5, 6}}, {}, 0), ({7: {9, 5}}, {}, 0), ({7: {9, 6}}, {}, 0), ({7: {0, 1, 2}}, {}, 0), ({7: {0, 1, 3}}, {}, 0), ({7: {0, 1, 5